In [ ]:
!pip install sentence-transformers faiss-cpu pandas groq

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from groq import Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
client = Groq(api_key="")

In [ ]:
file_path = "test_Matches_Data.csv"  # Replace with your actual file path in Google Colab
df = pd.read_csv(file_path)

# Check the data structure
print("Data Shape:", df.shape)
print(df.head())

Data Shape: (2537, 40)
   TEST Match No  Match ID                           Match Name  Series ID  \
0           2378   1183534    Australia Vs New Zealand 3Rd Test    1195334   
1           2383   1212542       Zimbabwe Vs Sri Lanka 2Nd Test    1212540   
2           2384   1213062      Pakistan Vs Bangladesh 1St Test    1195334   
3            650     63035         Pakistan Vs England 3Rd Test      60425   
4            651     63037  New Zealand Vs West Indies 2Nd Test      60426   

                                         Series Name Match Start Date  \
0    ICC World Test Championship  - 2019 (2019-2021)       2020-01-03   
1            Sri Lanka in Zimbabwe  - 2020 (2019/20)       2020-01-27   
2           Bangladesh in Pakistan  - 2020 (2019/20)       2020-02-07   
3         England tour of Pakistan  - 1969 (1968/69)       1969-03-06   
4  West Indies tour of New Zealand  - 1969 (1968/69)       1969-03-07   

  Match End Date Match Format  Team1 ID   Team1 Name  ...        Umpi

In [ ]:
df['text'] = df.astype(str).apply(' '.join, axis=1)

# Create a list of documents for embedding
documents = df['text'].tolist()
print("Sample Document:", documents[0])

Sample Document: 2378 1183534 Australia Vs New Zealand 3Rd Test 1195334 ICC World Test Championship  - 2019 (2019-2021) 2020-01-03 2020-01-06 TEST 2 Australia 34747 454 10 8 217.0 2.0 7.0 5 New Zealand 59148 256.0 10.0 11.0 136.0 9.0 4.0 Sydney Cricket Ground Sydney Australia Aleem Dar M Erasmus RB Richardson Australia bat Australia Australia won by 279 runs 78285.0 ['48739', '54337', '78285', '50281', '49024', '66190', '34747', '50781', '64244', '53330', '50724'] ['59148', '61839', '51094', '44930', '80639', '47492', '47231', '48641', '48432', '49219', '64947'] ['80639']


In [ ]:
model_name = "all-MiniLM-L6-v2"  # Using a lightweight model for embedding
embedder = SentenceTransformer(model_name)

# Generate embeddings for each document
document_embeddings = embedder.encode(documents, convert_to_tensor=False, show_progress_bar=True)
document_embeddings = np.array(document_embeddings)

print("Embedding Shape:", document_embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Embedding Shape: (2537, 384)


In [ ]:
# Create a FAISS index for similarity search
embedding_dim = document_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(document_embeddings)

# Save both the index and documents list in a .pkl file
with open("faiss_index.pkl", "wb") as f:
    pickle.dump((index, documents), f)

print("FAISS index and documents saved as 'faiss_index.pkl'")

FAISS index and documents saved as 'faiss_index.pkl'


In [ ]:
with open("faiss_index.pkl", "rb") as f:
    index, documents = pickle.load(f)

print("FAISS index and documents loaded successfully")


FAISS index and documents loaded successfully


APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama3-70b-8192` in organization `org_01jb6drm5hfnvtp6rt8d2j31j1` on tokens per minute (TPM): Limit 6000, Requested 328823, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
# def llama_generate_with_groq(query, top_k=5):
#     # Step 7a: Encode the query
#     query_embedding = embedder.encode([query], convert_to_tensor=False)

#     # Step 7b: Retrieve top-k similar documents
#     _, indices = index.search(np.array(query_embedding).astype("float32"), top_k)
#     retrieved_texts = [documents[i] for i in indices[0]]

#     # Step 7c: Combine retrieved contexts into a single prompt
#     context = " ".join(retrieved_texts)
#     input_text = f"{query} Context: {context}"

#     # Step 7d: Use Groq API to generate a response using Llama 3.1 model
#     completion = client.chat.completions.create(
#         model="llama3-70b-8192",
#         messages=[{"role": "user", "content": input_text}],
#         temperature=1,
#         max_tokens=1024,
#         top_p=1,
#         stream=True,
#         stop=None,
#     )

#     # Step 7e: Collect and print response
#     generated_text = ""
#     for chunk in completion:
#         generated_text += chunk.choices[0].delta.content or ""
#     return generated_text

# # Test the RAG function with Groq
# query = "Please tell me all the matches played in year 2020"
# print("Generated Answer:", llama_generate_with_groq(query))

Generated Answer: Based on the provided data, there are no matches played in the year 2020. All the matches mentioned are from the years 1986, 1992, 1993, 1997, and 1999.


In [ ]:
import numpy as np
import logging
import pandas as pd
import re
from collections import Counter

# Configure logging
logging.basicConfig(level=logging.INFO)

# Assume 'df' is your DataFrame loaded from the CSV
# df = pd.read_csv('your_dataset.csv')  # Uncomment this line to load your actual data

def get_total_runs_scored(df, year):
    """Get total runs scored in a specific year."""
    try:
        total_runs = df[df['Match Start Date'].str.contains(year)][
            ["Innings1 Team1 Runs Scored", "Innings2 Team1 Runs Scored",
             "Innings1 Team2 Runs Scored", "Innings2 Team2 Runs Scored"]
        ].sum().sum()
        return f"Total runs scored in {year}: {total_runs}"
    except Exception as e:
        logging.error(f"Error calculating total runs for year {year}: {e}")
        return "Error calculating total runs."

def get_most_man_of_the_match(df):
    """Get the player with the most Man of the Match awards."""
    try:
        valid_mom = df["MOM Player"].dropna().astype(str)
        most_common_mom = Counter(valid_mom).most_common(1)
        if most_common_mom:
            player_id, count = most_common_mom[0]
            return f"Player with the most Man of the Match awards: {player_id}, Awards: {count}"
    except Exception as e:
        logging.error(f"Error retrieving Man of the Match: {e}")
    return "No data available for Man of the Match."

def get_highest_score(df):
    """Get the highest score in Test cricket."""
    try:
        highest_score_team1 = df["Innings1 Team1 Runs Scored"].max()
        highest_score_team2 = df["Innings1 Team2 Runs Scored"].max()
        if highest_score_team1 >= highest_score_team2:
            match_row = df[df["Innings1 Team1 Runs Scored"] == highest_score_team1]
            match_info = match_row.iloc[0]
            return f"Highest score in Test cricket is {highest_score_team1} by {match_info['Team1 Name']} in the match {match_info['Match Name']}."
        else:
            match_row = df[df["Innings1 Team2 Runs Scored"] == highest_score_team2]
            match_info = match_row.iloc[0]
            return f"Highest score in Test cricket is {highest_score_team2} by {match_info['Team2 Name']} in the match {match_info['Match Name']}."
    except Exception as e:
        logging.error(f"Error retrieving highest score: {e}")
    return "No data available for highest score."

def get_lowest_score(df):
    """Get the lowest score in Test cricket."""
    try:
        lowest_score_team1 = df["Innings1 Team1 Runs Scored"].min()
        lowest_score_team2 = df["Innings1 Team2 Runs Scored"].min()
        if lowest_score_team1 <= lowest_score_team2:
            match_row = df[df["Innings1 Team1 Runs Scored"] == lowest_score_team1]
            match_info = match_row.iloc[0]
            return f"Lowest score in Test cricket is {lowest_score_team1} by {match_info['Team1 Name']} in the match {match_info['Match Name']}."
        else:
            match_row = df[df["Innings1 Team2 Runs Scored"] == lowest_score_team2]
            match_info = match_row.iloc[0]
            return f"Lowest score in Test cricket is {lowest_score_team2} by {match_info['Team2 Name']} in the match {match_info['Match Name']}."
    except Exception as e:
        logging.error(f"Error retrieving lowest score: {e}")
    return "No data available for lowest score."

def count_matches_per_team(df, team_name):
    """Count the number of matches played by a specific team."""
    try:
        matches_count = df[(df['Team1 Name'] == team_name) | (df['Team2 Name'] == team_name)].shape[0]
        return f"{team_name} has played {matches_count} matches."
    except Exception as e:
        logging.error(f"Error counting matches for {team_name}: {e}")
    return "No data available for the match count."

def process_dynamic_query(df, query):
    """Process the query to retrieve relevant information from the DataFrame."""
    # Normalize query for easier checking
    query = query.lower()

    if "total runs scored" in query:
        year_match = re.search(r"\b\d{4}\b", query)
        if year_match:
            year = year_match.group(0)
            return get_total_runs_scored(df, year)

    elif "most man of the match" in query:
        return get_most_man_of_the_match(df)

    elif "highest score" in query:
        return get_highest_score(df)

    elif "lowest score" in query:
        return get_lowest_score(df)

    elif "count of matches" in query or "matches played" in query:
        team_match = re.search(r"\b[a-zA-Z\s]+\b", query)  # Match team names
        if team_match:
            team_name = team_match.group(0).strip()
            return count_matches_per_team(df, team_name)

    # Add more queries as needed
    return "Sorry, I can't process that query."

def llama_generate_with_groq(query, top_k=5, temperature=1, max_tokens=1024):
    """Generate a response using the Groq API with Llama."""
    try:
        # Step 1: Process the query dynamically to find relevant information in the DataFrame
        df_result = process_dynamic_query(df, query)

        # Prepare context based on the DataFrame results
        context = df_result if df_result else "No relevant data found."

        # Step 2: Prepare the input text for the Llama model
        input_text = f"{query}\n\nHere are the results from the data:\n{context}"

        # Step 3: Use Groq API to generate a response using the Llama model
        logging.info("Generating response using the Groq API...")
        completion = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[{"role": "user", "content": input_text}],
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            stream=True,
            stop=None,
        )

        # Step 4: Collect and print response
        generated_text = ""
        for chunk in completion:
            generated_text += chunk.choices[0].delta.content or ""

        return generated_text.strip()  # Clean up leading/trailing spaces

    except Exception as e:
        logging.error(f"An error occurred while generating the response: {e}")
        return "An error occurred while generating the response."

# Test the RAG function with Groq
query = "Please tell me the lowest score in Test cricket."
print("Generated Answer:", llama_generate_with_groq(query))

query2 = "How many matches has Team India Played?"
print("Generated Answer:", llama_generate_with_groq(query2))


Generated Answer: Not quite!

The lowest score in Test cricket is actually 26, achieved by New Zealand against England at Eden Park, Auckland, New Zealand on March 25, 1955.

The score of 0.0 you mentioned is likely a typo or an error, as it is not possible for a team to score 0.0 in a Test match.
Generated Answer: That's okay! It seems like I need more information to provide an accurate answer. Team India has played many matches across various formats like Tests, ODIs, and T20Is. To provide a specific answer, could you please clarify which format or time period you're referring to?

Here are some rough estimates:

* Test matches: Over 500 matches (since 1932)
* ODI matches: Over 1,000 matches (since 1974)
* T20I matches: Over 150 matches (since 2006)

Let me know if you'd like more specific information!


Generated Answer: I apologize, but it seems like there is no data provided to process. If you could provide the data or context about the matches you're referring to (e.g. cricket, football, hockey, etc.), I'd be happy to help you with the total matches won by India.


FileNotFoundError: [Errno 2] No such file or directory: 'matches.csv'

NameError: name 'retrieved_texts' is not defined

In [ ]:
queries = [
    "Tell me highest score by a team sri lanka"
]

for query in queries:
    print("Query:", query)
    print("Generated Answer:", llama_generate_with_groq(query))
    print("\n" + "="*50 + "\n")

Query: Tell me highest score by a team
Generated Answer: The highest score by a team in the provided context is 805, achieved by West Indies in the 3rd Test against England in 1950 at Trent Bridge, Nottingham.




In [ ]:
queries = [
    "Tell the highest score made by India",
]

for query in queries:
    print("Query:", query)
    print("Generated Answer:", llama_generate_with_groq(query))
    print("\n" + "="*50 + "\n")

Query: Tell the highest score made by India
Generated Answer: According to the provided data, the highest score made by India is 1175, which was achieved in the 4th Test against West Indies in 1974-75 at the Madras Cricket Club Ground in Chennai, India.




In [ ]:
print("Number of indexed embeddings:", index.ntotal)

Number of indexed embeddings: 2537


In [ ]:
print("Document count:", len(documents))
print("Embedding count:", len(document_embeddings))

Document count: 2537
Embedding count: 2537


In [ ]:
queries = [
    "Tell the total number of matches India won",
    "How many matches did India win in 2023?",
    "What is the highest score India achieved?",
    "Average score of India across all matches?",
]

for query in queries:
    print("Query:", query)
    try:
        generated_answer = llama_generate_with_groq(query)
        if generated_answer.strip():
            print("Generated Answer:", generated_answer)
        else:
            print("Generated Answer: No response generated.")
    except Exception as e:
        print("Error generating answer:", str(e))
    print("\n" + "="*50 + "\n")


Query: Tell the total number of matches India won
Generated Answer: I'm happy to help! However, I notice that there is no information provided about the year or the type of matches (e.g., cricket, football, etc.) India participated in. Could you please provide more context or clarify which year and type of matches you're referring to? That way, I can try to help you find the correct answer.


Query: How many matches did India win in 2023?
Generated Answer: Since there is no relevant information found for the specified year 2023, it means that the data for 2023 is not available or has not been recorded yet. Therefore, I cannot provide the number of matches won by India in 2023.


Query: What is the highest score India achieved?
Generated Answer: I'm happy to help!

However, I need more context to provide the correct answer. Could you please specify the format of cricket you're referring to (e.g., Test, ODI, T20I) and the highest score you're looking for (e.g., individual innings, team t